In [24]:
from sympy import Symbol, simplify, Integer

In [67]:
class SymbolicValue:
    def __init__(self, initial_value=0, symbolic=False):
        if symbolic:
            self.value = Symbol('x')
        else:
            self.value = initial_value

    def increment(self):
        self.value += 1
        self.value = simplify(self.value)

    def decrement(self):
        self.value -= 1
        self.value = simplify(self.value)

    def __str__(self):
        return str(self.value)

    
class BrainfuckSymbolicSolver:
    def __init__(self):
        self.tape = [0] * 30000  
        self.pointer = 0
        self.symbolic_state = {}  # to track symbolic values

    def interpret(self, code):
        # sym interpret
        for char in code:
            self.execute_command(char)

    def execute_command(self, char):
        # Execute a single Brainfuck command
        if char == '>':
            self.pointer += 1
        elif char == '<':
            self.pointer -= 1
        elif char == '+':
            self.symbolic_increment()
        elif char == '-':
            self.symbolic_decrement()
        # handel loops
    
    def symbolic_increment(self):
        if self.pointer not in self.symbolic_state:
            self.symbolic_state[self.pointer] = SymbolicValue()
        self.symbolic_state[self.pointer].increment()

    def symbolic_decrement(self):
        if self.pointer not in self.symbolic_state:
            self.symbolic_state[self.pointer] = SymbolicValue()
        self.symbolic_state[self.pointer].decrement()


    def optimize(self, code):
        optimized_code = ""
        self.pointer = 0
        self.symbolic_state = {}

        for char in code:
            if char in ['+', '-']:
                self.execute_command(char)
            elif char == '>':
                self.pointer += 1  # update the pointer for '>'
            elif char == '<':
                self.pointer -= 1  # update the pointer for '<'
            elif char == '.':

                optimized_code += self.apply_optimizations()
                # Reset state after optimizations
                # self.pointer = 0
                # self.symbolic_state = {}
                # Add the print command
                optimized_code += f"sys.stdout.write(chr(tape[{self.pointer}]))\n"
            else:
                optimized_code += self.apply_optimizations()
                self.pointer = 0  
                self.symbolic_state = {}  
                optimized_code += char

        
        optimized_code += self.apply_optimizations()

        return optimized_code

    def apply_optimizations(self):
        optimized_python_code = ""
        for position, symbolic_value in self.symbolic_state.items():
            
            simplified_value = symbolic_value.value

            if isinstance(simplified_value, Integer):
                optimized_python_code += f"tape[{position}] = {simplified_value}\n"
            elif isinstance(simplified_value, Symbol):
                optimized_python_code += f"tape[{position}] = {simplified_value}\n"
            else:
                optimized_python_code += f"tape[{position}] = {simplified_value}\n"

        return optimized_python_code
    
    def optimize_and_convert_to_python(self, bf_filename):
        # Read Brainfuck code from file
        # with open(bf_filename, 'r') as file:
        #     bf_code = file.read()
        bf_code = bf_filename
        # Optimize Brainfuck code
        optimized_python_code = self.optimize(bf_code)

        # Create Python file with the same name but with .py extension
        # py_filename = bf_filename.rsplit('.', 1)[0] + '.py'
        py_filename = "test.py"
        with open(py_filename, 'w') as file:
            file.write(self.generate_python_code(optimized_python_code))

        print(f"Python code generated: {py_filename}")
    
    def generate_python_code(self, optimized_python_code):
        python_code_template = """import sys

def main():
    tape = [0] * 30000  # Initialize tape
    pointer = 0
    
    {}
    print()  # New line after program execution

if __name__ == "__main__":
    main()
        """
        # indented_code = '\n'.join('\t' + line for line in optimized_python_code.splitlines())
        indented_code = optimized_python_code.replace('\n', '\n    ')
        return python_code_template.format(indented_code)
 


In [5]:
solver = BrainfuckSymbolicSolver()
solver.interpret("++>+-") 
# solver.optimize()

In [69]:
solver = BrainfuckSymbolicSolver()
solver.optimize_and_convert_to_python("+++>+<--.")

Python code generated: test.py


In [27]:
solver = BrainfuckSymbolicSolver()
optimized_code = solver.optimize("+++++>+++")
print("Optimized code:", optimized_code)

Optimized code: tape[0] = 5
tape[1] = 3



In [28]:
optimized_code = solver.optimize("++>+-")
print("Optimized code:", optimized_code)

Optimized code: tape[0] = 2
tape[1] = 0

